# Лекция 7: аттрибуты объектов, дескрипторы,  json и регулярные выражения

## Работа с аттрибутами объектов

* По-умолчанию каждый объект имеет словарь полученный от своего класса, в котором хранятся поля объекта.
* Словарь доступен как ObjectClass.\_\_dict\_\_.
* При чтении (value = obj.name) или записи (obj.name = value) по имени обращение автоматически происходит к объекту в словаре по ключу name.
* Поведение при работе с аттрибутами объекта (полями, доступными по имени) можно переопределять.

### Переопределение доступа к полям по имени

* \_\_getattr\_\_(self, name) - переопределить доступ к аттрибуту (a = x.name) при попытке чтения.
  * Вызывается только тогда, когда не удалось найти аттрибут обычными способами: не является полем текущего объекта или какого-либо класса в иерархии предков, не перекрыт раньше методом дескриптора или обязательной версией геттера аттрибутов (см. ниже).
  * Должен либо вернуть сформированное значение по указанному имени, либо вызвать AttributeError.
  * Важно: используется только, если поле не найдено по имени другим способом.

* \_\_setattr\_\_(self, name, value) - при попытке записи данных в аттрибут (x.name = a).
  * Вызывается при попытке присваивания, перекрывает обычный механизм присваивания (в отличие от \_\_getattr\_\_).
  * Для присваивания аттрибута текущему объекту над вызвать \_\_setattr\_\_() у базового класса (self.name = value не сработает).
  * Важно: в отличие от предыдущего \_\_getattr\_\_ перекрывает стандартный механизм.

* \_\_delattr\_\_(self, name) - удалить аттрибут по имени.
   * Реализовывать стоит только в том случае, если del x.name имеет смысл для текущего объекта.

* \_\_getattribute\_\_(self, name) - переопределить доступ к аттрибуту при чтении (для new-style classes).
  * Подобен \_\_getattr\_\_, но в отличие от него перекрывает обычный механизм безусловно.
  * В случае, если также определён \_\_getattr\_\_, то последний вызовется только если его явно вызвать из \_\_getattribute\_\_ или бросить исключение AttributeError в \_\_getattribute\_\_.
  * Для получения аттрибута у текущего объекта, во избежание рекурсии, надо вызывать \_\_getattribute\_\_ у базового класса.

### Определение property полей

Декоратор property позволяет определять виртуальные поля, задавая (или нет) отдельные методы в качестве геттера, сеттера, удаляющего, а также определить документацию.

In [1]:
class PseudoMeter:
    def __init__(self):
        self.__temperature = 0

    def __get_temperature(self):
        print("Asked for temperature")
        return self.__temperature

    def __set_temperature(self, value):
        print("Got new value")
        if not isinstance(value, int):
            raise TypeError("int value required")
        self.__temperature = value

    temperature = property(__get_temperature, __set_temperature)

meter = PseudoMeter()
meter.temperature = 10
print(meter.temperature)

Got new value
Asked for temperature
10


Это был простейший пример. Можно делать аккуратнее и, зависимости от конкретного случая, использовать данный декоратор различными способами.

In [2]:
class PseudoMeter:
    def __init__(self):
        self.__temperature = 0

    @property
    def temperature(self):
        print("Asked for temperature")
        return self.__temperature

meter = PseudoMeter()
print(meter.temperature)

Asked for temperature
0


Или так:

In [3]:
class PseudoMeter:
    def __init__(self):
        self.__temperature = 0

    @property
    def temperature(self):
        print("Asked for temperature")
        return self.__temperature

    @temperature.setter
    def temperature(self, value):
        print("Got new value")
        if not isinstance(value, int):
            raise TypeError("int value required")
        self.__temperature = value

meter = PseudoMeter()
meter.temperature = 10
print(meter.temperature)

Got new value
Asked for temperature
10


### Дескрипторы

* Дескриптор - некоторый объект, который способен переопределять доступ к себе специальными методами будучи аттрибутом другого объекта.
* Объект называется дескриптором, если определён хотя бы один из методов \_\_get\_\_, \_\_set\_\_, \_\_delete\_\_.
* При обращении к аттрибуту, в котором записан объект-дескриптор соответствующие методы этого объекта могут быть вызваны вместо методов класса-хозяина (в зависимости от того, как определён дескриптор)
* [3.x] Во-втором питоне дескрипторы работают только для new-style classes (унаследованы от object (для классов) и type (метаклассов)), в третьем этой разницы уже нет.  

#### Дескрипторы данных и не-данных

* Дескриптор данных (data descriptor) - если определены \_\_set\_\_ или \_\_delete\_\_.
* Дескриптор не-данных (non-data descriptor) - если определен \_\_get\_\_, но не \_\_set\_\_ и не \_\_delete\_\_.
* Дескриптор данных имеет приоритет перед поиском в словаре текущего объекта, в котором он находится.
* Дескриптор не-данных запрашивается только после того, как по имени в текущем объекте не было найдено аттрибутов.
* Read-only дескриптор данных - реализовать \_\_set\_\_, но выбрасывать из него AttributeError.

#### Протокол дескрипторов

* descr.\_\_get\_\_(self, instance, owner) --> value
  * Вызывается при обращении для чтения к аттрибуту класса-владельца (class attribute) или объекта-владельца (instance attribute).
  * instance - объект-владелец, для которого вызвано получение аттрибута (None, если вызвано для класса).
  * owner - класс-владелец.
  * Должен либо возвращать значение, либо бросать AttributeError.

* descr.\_\_set\_\_(self, instance, value) --> None
  * Вызывается для присваивания аттрибуту объекта-владельца instance значения value.

* descr.\_\_delete\_\_(self, instance) --> None
  * Вызывается для удаления аттрибута из объекта-владельца instance.

#### Ещё один метод

* object._\_set_name\__(self, owner, name)
  - Вызывается при создании класса, в котором будет жить дескриптор
  - Стандартная реализация присваивает self в name у owner.

#### Варианты вызова дескрипторов

* Напрямую - descr.\_\_get\_\_(instance, type(instance))
  * Обычно не используется.
* От объекта - obj.descr:
  * type(instance).\_\_dict\_\_['descr'].\_\_get\_\_(instance, type(instance))
* От класса - obj.descr
  * owner.\_\_dict\_\_['descr'].\_\_get\_\_(None, owner)
* От super - super(owner, instance).descr
  * Ищет в instance.\_\_class\_\_.\_\_mro\_\_ ближайший базовый класс-предок ownerBase и вызывает дескриптор.
  * Вызвает так: ownerBase.\_\_dict\_\_['descr'].\_\_get\_\_(instance, instance.\_\_class\_\_)
  

#### Замечания про дескрипторы и переопределение доступа к полям

* Преобразования из предыдущего списка автоматически делает \_\_getattribute\_\_.
* Если его переопределить, то больше не будет работать автоматическое использование дескрипторов.
* В зависимост от варианта вызова используется object.\_\_getattribute\_\_ (для объекта) или type.\_\_getattribute\_\_ (для класса).

#### Примеры дескрипторов в языке Python

Функции staticmethod, classmethod и property (также используемые как декораторы) реализованы с помощью протокола дескрипторов.

Пример property на чистом Python (в стандартном интерпретаторе реализована нативно):

In [4]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

#### Функции и методы

* ООП в Python основывается на использовании данных и функций внутри классов.
* Для объединения этих двух частей воедино используются дескрипторы не-данных.

Как это работает:

* В словарях полей классов методы хранятся просто как функции, реализованные через def и lambda.
* Единственное отличие - соглашение о том, что первый аргумент соответствует вызывающему объекту self.
* Все объекты-функции имеют \_\_get\_\_, что делает их дескрипторами не-данных.
* При доступе к объекту-функции в зависимости от того, как он произведён (объект-владелец или через класс-владелец) возвращает связанные методы или просто функцию (в [3.x]) соответственно.

In [5]:
class Owner:
    def func(self, value):
        return value

instance = Owner()
print(Owner.__dict__['func'])
print(Owner.func) # [3.x]
print(instance.func)

<function Owner.func at 0x7fe132fd50d0>
<function Owner.func at 0x7fe132fd50d0>
<bound method Owner.func of <__main__.Owner object at 0x7fe132fe95f8>>


Посмотрим ещё примеры реализации версий staticmethod и classmethod на чистом Python:

In [6]:
class StaticMethod:
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f

In [7]:
class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)

        def newfunc(*args):
            return self.f(klass, *args)

        return newfunc

## JSON

* JSON -  JavaScript Object Notation - формат передачи данных.
* Не зависит от языка - поддержка есть практически для всех языках программирования.
* Имеет человекочитаемый вид.
* Позволяет просто и понятно описывать структурированные данные.
* Значения: объект, массив, строка, число, true, false, null.

In [8]:
response_message = """
{
  "data": [
    {
      "type": "articles",
      "id": "1",
      "attributes": {
        "title": "JSON API paints my bikeshed!",
        "body": "The shortest article. Ever.",
        "created": "2015-05-22T14:56:29.000Z",
        "updated": "2015-05-22T14:56:28.000Z"
      },
      "relationships": {
        "author": {
          "data": {
            "id": "42",
            "type": "people"
          }
        }
      }
    }
  ],
  "included": [
    {
      "type": "people",
      "id": "42",
      "attributes": {
        "name": "John",
        "age": 80,
        "gender": "male"
      }
    }
  ]
}
"""

В языке Python есть модуль стандартной библиотеки для работы с JSON:

In [9]:
import json
dir(json)

['JSONDecodeError',
 'JSONDecoder',
 'JSONEncoder',
 '__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_default_decoder',
 '_default_encoder',
 'decoder',
 'dump',
 'dumps',
 'encoder',
 'load',
 'loads',
 'scanner']

C помощью этого модуля можно превратить json в объекты стандартные объекты языка Python.

In [10]:
# convert from json text to python data
data = json.loads(response_message)

from pprint import pprint
pprint(data)

{'data': [{'attributes': {'body': 'The shortest article. Ever.',
                          'created': '2015-05-22T14:56:29.000Z',
                          'title': 'JSON API paints my bikeshed!',
                          'updated': '2015-05-22T14:56:28.000Z'},
           'id': '1',
           'relationships': {'author': {'data': {'id': '42',
                                                 'type': 'people'}}},
           'type': 'articles'}],
 'included': [{'attributes': {'age': 80, 'gender': 'male', 'name': 'John'},
               'id': '42',
               'type': 'people'}]}


In [11]:
print(data.keys(), type(data))
print(data["data"][0]["attributes"])

dict_keys(['data', 'included']) <class 'dict'>
{'title': 'JSON API paints my bikeshed!', 'updated': '2015-05-22T14:56:28.000Z', 'created': '2015-05-22T14:56:29.000Z', 'body': 'The shortest article. Ever.'}


Или наоборот:

In [12]:
target_description = {
    "id": 100500,
    "name": "ahaha",
    "marks": [5, 7, 5, 8],
    "is_valid": True
}

In [13]:
print(json.dumps(target_description))

{"name": "ahaha", "is_valid": true, "marks": [5, 7, 5, 8], "id": 100500}


In [14]:
print(json.dumps(target_description, indent=4))

{
    "name": "ahaha",
    "is_valid": true,
    "marks": [
        5,
        7,
        5,
        8
    ],
    "id": 100500
}


Кроме этого:

* Аналогично есть функции load/dump, которые вместро строк работают с file-like объектами.
* Можно унаследоваться от JSONEncoder/JSONDecoder и определить свои правила кодирования-декодирования json с помощью функций loads/load/dump/dumps.

Подробнее про json:

* http://www.json.org/  
* https://en.wikipedia.org/wiki/JSON  
* https://docs.python.org/3.5/library/json.html  

### Регулярные выражения

* Регулярное выражение - формальный язык поиска и манипуляций над текстом, представлен грамматикой со специальными метасимволами, задающими определённые правила преобразования над последовательностями символов.
* Шаблон (pattern) - совокупность символов и метасимволов задающее правила поиска и преобразований в его процессе.
* В стандартной библиотеке Python есть библиотека для удобной работы с регулярными выражениями.
* Библиотека называется re.

Рассмотрим простые примеры использования библиотеки для работы с регулярными выражениями:

In [15]:
import re
print(dir(re))

['A', 'ASCII', 'DEBUG', 'DOTALL', 'I', 'IGNORECASE', 'L', 'LOCALE', 'M', 'MULTILINE', 'S', 'Scanner', 'T', 'TEMPLATE', 'U', 'UNICODE', 'VERBOSE', 'X', '_MAXCACHE', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', '_alphanum_bytes', '_alphanum_str', '_cache', '_cache_repl', '_compile', '_compile_repl', '_expand', '_locale', '_pattern_type', '_pickle', '_subx', 'compile', 'copyreg', 'error', 'escape', 'findall', 'finditer', 'fullmatch', 'match', 'purge', 'search', 'split', 'sre_compile', 'sre_parse', 'sub', 'subn', 'sys', 'template']


In [16]:
import re

pattern = "abc"
match = re.match(pattern, "abc")
print(match.pos)

0


In [17]:
print(re.match("abc", "abcd"))

<_sre.SRE_Match object; span=(0, 3), match='abc'>


In [18]:
print(re.match("^abc$", "abc"))
print(re.match("^abc$", "abcd"))

<_sre.SRE_Match object; span=(0, 3), match='abc'>
None


In [19]:
print(re.match("abc", "dabc"))
print(re.search("abc", "dabc"))

None
<_sre.SRE_Match object; span=(1, 4), match='abc'>


In [20]:
help(re.match)

Help on function match in module re:

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a match object, or None if no match was found.



In [21]:
help(re.search)

Help on function search in module re:

search(pattern, string, flags=0)
    Scan through string looking for a match to the pattern, returning
    a match object, or None if no match was found.



In [22]:
print(re.match("a[0-9]b", "a1b"))

<_sre.SRE_Match object; span=(0, 3), match='a1b'>


In [23]:
print(re.match("a[0-9]*b", "a1124234234b"))

<_sre.SRE_Match object; span=(0, 12), match='a1124234234b'>


In [24]:
print(re.match("a[0-9]*b", "ab"))
print(re.match("a[0-9]+b", "ab"))

<_sre.SRE_Match object; span=(0, 2), match='ab'>
None


In [25]:
match = re.match("(.+)@(.+\.com)", "user@domain.com")
print(match.group())
print(match.group(1))

user@domain.com
user


Подробнее про регулярные выражения:

* https://ru.wikipedia.org/wiki/Регулярные_выражения  
* https://ru.wikibooks.org/wiki/Регулярные_выражения  
* https://docs.python.org/3.5/library/re.html  
* https://developer.mozilla.org/en-US/docs/Web/JavaScript/Guide/Regular_Expressions  
* http://www.regexr.com/  
* http://habrahabr.ru/post/115825/
* Книга "Mastering Regular Expressions", by Jeffrey Friedl  
* https://swtch.com/~rsc/regexp/regexp1.html  
* https://swtch.com/~rsc/regexp/regexp2.html  
* https://swtch.com/~rsc/regexp/regexp3.html  

Важно понимать, когда стоит употреблять:

In [26]:
print("""Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems.""")

Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems.


* https://blog.codinghorror.com/regular-expressions-now-you-have-two-problems/  
* https://programmers.stackexchange.com/questions/113237/when-you-should-not-use-regular-expressions  
* https://blog.codinghorror.com/regex-use-vs-regex-abuse/  

Полезно на будущее для самостоятельного чтения:
* Для понимания языков программирования, низкоуровневых идей в основе парсеров и того, как работают регулярные выражения: книга "Введение в теорию автоматов, языков и вычислений.". Авторы: Джон Хопкрофт, Раджив Мотвани, Джеффри Ульман.
